Na željo enega od študentov, nekaj o lambda funkcijah. Vendar bi rad naredil kaj več kot pokazal, kako jih definiramo. To je preprosto; rad bi povedal nekaj o tem, kaj lambda funkcije pravzaprav so.

### Kaj sta izraz in stavek -- in kaj ni izraz

Preden začnemo, moramo povedati nekaj o izrazih (*expression*) in stavkih (*statement*).

Izraz je nekaj, kar lahko izračunamo. Kolegi se bodo kremžili in me prezirali, vendar bomo razložili zelo po domače: izraz je nekaj, kar lahko postavimo desno od enačaja, uporabimo kot argument funkcije, ... in seveda še kje.

In [38]:
a = 2 * 2
b = [1, 2, 3] * 5
pow(2 + 3, len("Benjamin") - 2)

15625

Tule so `2 * 2`, `[1, 2, 3] * 5`, `2 + 3` in `len("Benjamin") - 2` izrazi.

Stavek pa Wikipedia lepo razloži kot "*a statement is a syntactic unit of an imperative programming language that expresses some action to be carried out.*". Primer je recimo `return`, `import`, pa `if` z vso svojo vsebino (ki lahko vključuje druge stavke in izraze) ali, recimo, prireditveni stavek (štiri vidimo zgoraj).

V nekem smislu je vsak izraz tudi stavek. Obratno pa ne, vsaj ne v Pythonu. Returna ali importa ne moremo postaviti desno od enačaja, saj to nima smisla.

Eden od stavkov je tudi `def`; ta *imenu* priredi *funkcijo*.

In [39]:
def f(x):
    return x * 2

Tu smo sestavili funkcijo (ki vsebuje stavek `return`, ki vsebuje izraz `x * 2`, a to ni zelo pomembno) in tej funkciji dali ime `f`.

Ker je `def` stavek in ne izraz, ne moremo pisati

```
f = def (x):
    return x * 2
```

čeprav stavek `def` v nekem smislu dela prav to.

Na prvi pogled: funkcijo v Pythonu se pač definira z `def f(x):` in ne `f = def (x)` -- kaj potem. Gre samo za sintaktični detajl. O, ko bi bilo vsaj tako.

Preden gremo naprej, si samo zapomnimo:

```
def (x):
    return x * 2
```

bi bil *izraz*, ki bi definiral funkcijo, ki vrne dvakratnik podanega argumenta. Vendar Python nima takšne sintakse.

### Motivacija: argument za `max`, `min` in `sorted` (in `sort`)
Imamo seznam nizov

In [40]:
s = ["Ančka", "Berta", "Cecilija", "Dani"]

Radi bi dobili najdaljše ime. In najkrajšega.

In [41]:
max(s)

'Dani'

In [42]:
min(s)

'Ančka'

Seveda ne. Da, hočemo "maksimalno" in "minimalno" ime, vendar je ključ, po katerem bomo primerjali imena, njihova dolžina. Lahko bi napisali funkcijo `max_by`, ki bi poleg seznama dobila še funkcijo, ki kot dodatni argument dobi ključ.

In [43]:
def max_by(s, key):
    m = None
    for x in s:
        if m is None or key(x) > key(m):
            m = x
    return m

In [44]:
max_by(s, len)

'Cecilija'

Morda je komu nenavadno, da smo funkciji kot argument podali funkcijo. To ni nič takega, vsaj ne v spodobnih jezikih.

Naš `max_by` deluje tako, kot vsako iskanje največjega števila, ki smo se ga šli že velikokrat, le namesto `x > m` pišemo `key(x) > key(m)`: namesto, da bi vrnila največji element, vrne tisti element, za katerega funkcija `key` vrne največjo vrednost. Če kot ključ podamo `len`, bo pač vrnila element, za katerega `len` vrne največjo vrednost. Torej najdaljši niz.

Dobra novica: takšne funkcije nam ni potrebno napisati. Funkcije `max`, `min`, `sorted` in `sort` že sprejemajo takšen argument, le da ga moramo podati kot poimenovan argument.

In [45]:
max(s, key=len)

'Cecilija'

In [46]:
min(s, key=len)

'Dani'

In [47]:
sorted(s, key=len)

['Dani', 'Ančka', 'Berta', 'Cecilija']

Slednja, `sorted` uredi elemente seznama po vrednosti, ki jo vrača `len`.

Takole pa bi uredili števila po njihovi absolutni vrednosti, se pravi z ignoriranjem predznaka.

In [48]:
sorted([3, 6, -8, 1, -3, 4, -5], key=abs)

[1, 3, -3, 4, -5, 6, -8]

Tule je še bolj zanimiv primer: imamo slovar.

In [49]:
d = {"Ana": 15, "Berta": 30, "Cilka": 23}

Dobiti želimo ključ, ki mu pripada največja vrednost.

In [50]:
max(d, key=d.get)

'Berta'

Seveda to dela tudi z našim `max_by`:

In [51]:
max_by(d, d.get)

'Berta'

Kako to deluje? Tudi `d.get` je funkcija. Predstavljajmo si našo funkcijo, v kateri je `s` v resnici `d` in `key` v resnici `d.get`:

```python
    m = None
    for x in d:
        if m is None or d.get(x) > d.get(m):
            m = x
```

Zanka `for x in d` gre čez ključe in v `if` primerjamo vrednosti, ki pripadata ključema `x` in `m`.

Deluje celo

In [52]:
sorted(d, key=d.get)

['Ana', 'Cilka', 'Berta']

Dobili smo seznam ključev (ker `sorted` vedno vrača seznam reči, ki jih dobi pri iteriranju prek podanega argumenta), urejene po njihovih vrednostih (ker to pač vrača `d.get`)

### Ključ, ki zahteva novo funkcijo

Imejmo seznam številk.

In [53]:
t = [4, 8, 14, 2, 11, 15]

Dobiti želimo število, ki je najbližje 10. Za to bomo poklicali `min(s, key=...)` vendar nekako nimamo pri roki funkcije, ki bi izračunala razliko med številom in 10.

So duhoviteži, ki bi napisali

In [54]:
10 + min((x - 10 for x in t), key=abs)

11

vendar ne pustimo, da nam pokvarijo primer. :)

Če funkcije, ki izračuna razliko med številom in 10 ni, jo pač lahko napišemo.

In [55]:
def razlika10(x):
    return abs(x - 10)

min(t, key=razlika10)

11

To sicer deluje, kot mora, vendar pokvari uporabnost `min`, saj zahteva, da definiramo novo funkcijo samo zato, da jo uporabimo v enem brezzveznem `min`. Če je tako, da ona "duhovita" rešitev zgoraj pravzaprav boljša.

### Zakaj se to ne da?

Osnovni problem smo pokazali v začetku. Recimo, da bi lahko funkcije v Pythonu definirali z

```
razlika10 = def (x):
    return abs(x - 10)
```

Z drugimi besedami, recimo, da bi bila

```
def (x):
    return abs(x - 10)
```

funkcija, ki vrne razliko med x in 10 -- v smislu "izraz, ki vrne takšno funkcijo".

Če bi bilo to tako (vendar ni), bi lahko napisali

```
min(s, def (x):
    return abs(x - 10)
```

ali, morda, brez nove vrste

```
min(s, def (x): return abs(x - 10)
```

Vendar ne moremo.

### Lambda-funkcije

Pravzaprav lahko, le sintaksa je malo drugačna: namesto `def` moramo napisati `lambda`, okrog `x` ne naredimo oklepajev in `return` izpustimo.

In [56]:
razlika10 = lambda x: abs(x - 10)

In [57]:
razlika10(6)

4

Potem je `razlika10` funkcija, kot katerakoli druga. Če funkcijo `razlika10` definiramo z lambdo, kot tu, ali z `def`, dobimo praktično enako funkcijo. (S par malenkostnimi razlikami, o katerih tu ne bi razpravljali.) Preden se razvnamete, moram samo povedati, da lambd ne uporabljamo zato, da bi se izognili `def` in tlačili reči v eno vrstico. Lambde praviloma uporabljamo takrat, ko potrebujemo funkcijo, ki jo želimo podati kot argument.

In [58]:
min(t, key=lambda x: abs(x - 10))

11

In [59]:
sorted(t, key=lambda x: abs(x - 10))

[11, 8, 14, 15, 4, 2]

Takole sortiramo nize po abecedi, če bi jih brali z desne proti levi:

In [62]:
sorted(s, key=lambda x: x[::-1])

['Cecilija', 'Ančka', 'Berta', 'Dani']

In takole po številu `i`-jev.

In [63]:
sorted(s, key=lambda x: x.count("i"))

['Ančka', 'Berta', 'Dani', 'Cecilija']

Padajoče? Prav.

In [64]:
sorted(s, key=lambda x: -x.count("i"))

['Cecilija', 'Dani', 'Ančka', 'Berta']

Ali pa, seveda,

In [65]:
sorted(s, key=lambda x: x.count("i"), reverse=True)

['Cecilija', 'Dani', 'Ančka', 'Berta']

### Kaj je torej lambda

**Lambda je *izraz*, ki vrne funkcijo.** S poudarkom na izraz. Ker je izraz, lahko njegovo vrednost priredimo spremenljivki ali pa jo podamo kot argument.

### Kaj lambde znajo in česa ne

Kaj znajo in česa ne? Predvsem slednje. :)

Torej: prejmejo lahko poljubno število argumentov, argumente s privzetimi vrednostmi in tako naprej. Ta del štima.

Problem je v tem, kaj lahko vsebujejo. En sam izraz. Lambde so kot funkcije, ki vsebujejo le en `return`. Izraz je lahko poljubno dolg in zapleten, vendar le izraz. Nobenih pogojnih stavkov, zank (lahko pa imajo generatorje, izpeljane sezname in tako naprej), da o kakih importih ne govorimo.

Za Pythonove lambde bi lahko kdo rekel, da so sramota za jezik. Vsi "normalni" jeziki, ki imajo lambda-funkcije, dopuščajo, da te vsebujejo vse, kar vsebujejo "ne-lambde". Javascript, recimo, pusti, da funkcijo za izračun vsote zapišemo kot

```
function sum(a) {
    let v = 0;
    for(let x of a) {
        v += x;
    }    
    return v;
}
```

ali kot

```
sum = function (a) {
    let v = 0;
    for(let x of a) {
        v += x;
    }    
    return v;
}
```

ali, s še imenitnejšo novejšo notacijo

```
sum = (a) => {
    let v = 0;
    for(let x of a) {
        v += x;
    }    
    return v;
}
```

Če je to, kar računa funkcija, možno zapisati z enim samim izrazom (torej: če je funkcija tako preprosta, da bi jo bilo mogoče stlačiti celo v ubogo Pythonovo lambdo), pa lahko pišemo celo

```
f = (x) => 2 * x
```

ali, če je argument le eden,

```
f = x => 2 * x
```

Tu piše *f je funkcija, ki prejme x in vrne 2 * x*.

Ko programiram v Javascriptu, praktično nikoli ne pišem `function`, temveč prav vse funkcije definiram kot (v bistvu) lambde.

V jeziku Kotlin pa so lambde in njihova uporaba čisti "next level".

Bi imel torej ta, ki bi rekel, da so lambde v Pythonu ena velika sramota, prav? Noben jezik ne more vsega. Python ima pregledno sintakso, v kateri so bloki določeni z dvopičjem in zamikom, ne s kakimi zavitimi oklepaji ali čem podobnim. To enim ni všeč, večini pa je. Po drugi strani pa je v tej sintaksi težko opisati lambde, ki bi vsebovale kaj več kot posamičen izraz. Tiste, ki so na sveže prišli iz jezikov z mogočnimi lambdami, to seveda moti. Osebno sem vajen tako Javascripta kot Pythona in v slednjem lambd ne pogrešam tako pogosto. Delno ga rešuje, da ima tako lepo sintakso za generatorje in izpeljane sezname -- to so namreč prav situacije, v katerih v drugih jezikih najpogosteje potrebujemo lambde.